In [1]:
import re
import os
import pandas as pd
import numpy as np
import tqdm

In [2]:
from datautils import data_dialog
from transformers import BertTokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.add_special_tokens({'sep_token': '__eou__'})

1

In [4]:
# data = data_dialog.DialogData("./data/reddit_1M/train_dialogues.txt", tokenizer)
def load_dialogs(path):
    dialogs = []
    with open(path) as f:
        for line in tqdm.auto.tqdm(f, desc="Loading data"):
            # if len(self.data) > max_items:
            #     break  # Enough for now
            Full_D = line.strip().strip("__eou__").split(" __eou__ ")
            if len(Full_D) >= 2:
                dialogs.append(Full_D)
#                 for j in range(2, len(Full_D) + 1):
#                     D = Full_D[:j]
#                     C = " __eou__ ".join(D[:-1]).strip()
#                     R = D[-1].strip()
                    # mid = len(D)//2
                    # C = " __eou__ ".join(D[:mid])
                    # R = " __eou__ ".join(D[mid:])
    return dialogs

In [5]:
data_dir = "./data/reddit_1M/"
# use glob.glob to search
data_file = "train_dialogues.txt"

In [65]:
train_dialogs = load_dialogs(data_dir + data_file)
# train_dialogs = load_dialogs("./data/dailydialog/dialogues_train.txt")

In [72]:
ext1 = r"io|com|org|net|us|co|biz|info|jobs|mobi|name|ly|tel|kitchen|email|tech|estate|xyz|codes|bargains|bid|expert|int|mil|edu|gov|ca|cn|fr|ch|au|in|de|jp|nl|uk|mx|no|ru|br|se|es"
extensions = r"gt|cy|kg|jobs|coop|ag|li|pa|qa|uz|ly|al|so|mn|cr|dz|md|bz|travel|im|la|ke|lu|ba|do|uy|name|mk|ve|xyz|club|is|eg|bd|cat|ec|pw|am|nu|ma|ge|sa|asia|lk|to|fm|kz|xxx|country|pro|tk|ng|ee|th|si|xn--p1ai|ws|mobi|ph|lv|ae|rs|hr|pe|az|su|bg|gov|pk|lt|by|my|ie|nz|sg|hk|io|cl|il|pt|cc|fi|sk|id|no|dk|be|us|ar|at|ch|hu|me|biz|edu|se|vn|ro|mx|tr|tv|za|eu|co|cz|ua|tw|kr|ca|gr|es|nl|au|ir|info|cn|fr|it|pl|in|uk|br|jp|de|org|ru|net|com"

In [73]:
url_reg = re.compile(r"([a-zA-Z0-9]+://)?([\w]{2,}\.){1,}(" + ext1 + r"|" + extensions + r")([/?]([^ \n()\'\"]+))?")

In [78]:
word_reg = re.compile(r"[ ]+")
# url_reg = re.compile(r"([a-zA-Z0-9]+://)?([\w]{2,}\.){1,}(io|com|org|net|us|co|biz|info|jobs|mobi|name|ly|tel|kitchen|email|tech|estate|xyz|codes|bargains|bid|expert|int|mil|edu|gov|ca|cn|fr|ch|au|in|de|jp|nl|uk|mx|no|ru|br|se|es)([/?]([^ \n()\'\"]+))?")
post_title_regex = re.compile(r"<selfbr>")

stats = []
filtered_data = []
urls = []
for d_raw in tqdm.auto.tqdm(train_dialogs[:6000]):
    if len(d_raw) < 2:
        continue
    d = " __eou__ ".join(d_raw)

    # selfbr removal (comes after title of the post)
    d = post_title_regex.sub(":", d)
    
    wd = word_reg.split(d)
    
    # first utterance
    first_utt_len = len(word_reg.split(d_raw[0]))
    
    
    
    # url filter
    L = list(url_reg.finditer(d))
    d = url_reg.sub("[URL]", d)
    if L is not None:
        urls.extend([match.group() for match in L])

    # length filter
    ll_utt = [1 for u in d.split(" __eou__ ") if len(u.split(" ")) > 80]
    avg_utt_len = np.mean([len(u.split(" ")) for u in d.split(" __eou__ ")])
    
    # if avg_utt_len > 60:
    #     print(d.replace("__eou__", "\n\n>>\t"))
    #     print("=====================================\n")
    this_stat = {
        'urls': len(L),
        'avg_utt_len': avg_utt_len,
        'very_long_utt': len(ll_utt),
        'turns': len(d.split("__eou__")),
        'first_utt_len': first_utt_len
    }
    stats.append(this_stat)
    
    if this_stat['urls'] < 3 and this_stat['avg_utt_len'] <= 50 and \
        this_stat['avg_utt_len'] > 3 and this_stat['first_utt_len'] > 4 and \
        this_stat['very_long_utt'] == 0:
        filtered_data.append(d + " __eou__")

In [79]:
print(f"salvaged {len(filtered_data)} dialogs")

salvaged 2584 dialogs


In [80]:
df_stats = pd.DataFrame(stats)

In [81]:
df_stats.describe()

,urls,avg_utt_len,very_long_utt,turns,first_utt_len
count,6000.000000,6000.000000,6000.00000,6000.000000,6000.000000
mean,0.716333,55.614190,0.83050,4.801500,108.532333
std,3.449954,62.008011,1.05386,1.313285,204.931811
min,0.000000,1.166667,0.00000,4.000000,1.000000
25%,0.000000,21.191667,0.00000,4.000000,16.000000
50%,0.000000,38.750000,1.00000,4.000000,55.000000
75%,1.000000,67.148810,1.00000,5.000000,118.250000
max,168.000000,1065.750000,8.00000,30.000000,4598.000000


In [82]:
df_stats.loc[df_stats.avg_utt_len > 50].shape

(2270, 5)

In [83]:
df_stats.loc[df_stats.avg_utt_len < 4].shape

(23, 5)

In [84]:
df_stats.loc[df_stats.urls > 2].shape

(365, 5)

In [85]:
df_stats.loc[df_stats.very_long_utt > 0].shape

(3112, 5)

In [86]:
df_stats.loc[df_stats.first_utt_len < 4].shape

(137, 5)

In [87]:
try:
    os.makedirs(os.path.join(data_dir, "filtered/"))
except FileExistsError:
    pass

In [88]:
with open(os.path.join(data_dir, "filtered/", data_file), "w") as outf:
    for line in filtered_data:
        outf.write(line + "\n")

In [90]:
# urls